Previous: [Xu, Hong, and Qin's Methodology](06.ipynb) | [Table of Contents](00.ipynb) | Next: [The Willow Tree: A Brief Recap](08.ipynb)

# The Willow Tree Lattice Handbook

## Applications to Derivatives Pricing

To many problems in financial engineering, numerical methods are the only viable solution. Relying on a solid algorithm is key for market participants, because the quality of the estimates makes the difference between profit and loss.

A valuation procedure leads to acceptable results if and only if the adopted lattice is valid. A valid lattice is one that generates estimates of increasing precision when its structure gets finer and finer. In the limit, these estimates should converge to the true, often unknown price of the derivative.

### The Willow Tree Is A Valid Lattice

The willow tree achieves convergence to the true price of the derivative by focusing on the standard Brownian motion, the driver of the stock price process and the only source of risk in the Black-Scholes-Merton environment.

For example, the value of a European, plain vanilla call option depends on the final price of the stock. This, in turn, is a function of $W(T)$, the value of the standard Brownian motion at the expiry date:

$$
\begin{align*}
c = \bigg[&\boxed{S_T}\mathcal{N}(d_1)-K\mathcal{N}(d_2)\bigg]\cdot e^{-rT}\nonumber\\
&\ S_T=S_0\exp{\bigg\{\Big(r-\frac{1}{2}\sigma^2\Big)T+\sigma \boxed{W(T)}\ \bigg\}}
\end{align*}
$$

Yet, for an accurate valuation of derivatives, the proxy for the standard Brownian motion must be reliable. In the willow tree lattice framework, this occurs if and only if the discrete Markov process $\big\{Y_{t_k}\big\}_{k=0}^n$ (see [The Discrete Process Y](04.ipynb)) converges to the desired distribution.

As conjectured by Ho [8], and later proven by Haussmann and Yan [7], $\big\{Y_{t_k}\big\}_{k=0}^n$ meets such requirement when the number of space nodes goes to infinity, $m\to\infty$, and the length of time increments goes to zero, $h\to0$:

$$
\big\{Y_{t_k}\big\}_{k=0}^n\ \to\ W(t),\qquad m\to\infty,\quad h\to0
$$

If the equation holds for $Y_{t_k}$, for each time step, all suitable mappings of this process must also converge to their respective distributions. For example, for an adequately fine grid, the stock price process $S_{t_k}=S\big(Y_{t_k}\big)$ must follow the geometric Brownian motion, for all $t_k$:

$$
S_{t_k}=S_0\exp{\bigg\{\Big(r-\frac{1}{2}\sigma^2\Big)t_k+\sigma Y_{t_k}\bigg\}}\ \to\ S(t)=S_0\exp{\bigg\{\Big(r-\frac{1}{2}\sigma^2\Big)t_k+\sigma W(t)\bigg\}}
$$

The latter equation is particularly relevant: it ensures that the willow tree is a fruitful pricing technique, and justifies its use in derivative valuation.

## Lattice-Based Valuation of Derivatives

The procedure to value equity derivatives in a lattice framework involves three steps [9].

- The first step, the forward part of the algorithm, is to build a grid that collects the set of possible realisations of the stock price process in the time interval $[0,T]$:

 $$
 \Big\{S\big(t_k\big)\Big\}_{k=0}^n
 $$

 Process $S$ can give rise to either an increasing (binomial) or a constant (willow) number of observations in time. The size of the grid will be a function of both the model selected and the number of time steps.


- The second step is to find the payoffs of the derivative at each final node of the lattice, applying a suitable mapping $\phi$ to the stock price realisations in $T$:

 $$
 S(T)\quad\Longrightarrow\quad X(T)=\phi\big[S(T)\big]
 $$

 For instance, $X(T)=\big(S(T)-K\big)^+$ for a plain vanilla call option, or $X(T)=\max_{0\leq s\leq T}S(s)-S(T)$ for a lookback put option.


- The last step, the backward part of the algorithm, is to compute the price of the derivative at valuation date, $t_0=0$. The starting point is the end of the lattice, because at that point the values of the instrument are known, and equal to the payoffs:

 $$
 V^X(T)=X(T)
 $$

 At each previous date $t_k=T-1,...,0$, and for all nodes, the values are found by iteratively computing the risk-neutral conditional expectation of discounted future payoffs. For European-type derivatives, for which there is no intermediate exercise, the value is computed according to:

 $$
 V^X(t_k)=B(t_k)\cdot\mathbb{E}^Q\Bigg[\frac{V^X(t_{k+1})}{B(t_{k+1})}\ \Bigg\vert\ \mathcal{P}_{t_k}\Bigg]
 $$

 with $\mathcal{P}_{t_k}$ the set of information available at time $t_k$, and $B(\cdot)$ the money market account, the numéraire under the risk-neutral measure, $Q$:

 $$
 B(t_k)=\exp\bigg\{\int_0^{t_k}r(s)ds\bigg\}
 $$

 For American-type derivative instruments, which can be exercised at every intermediate date, it is important to check, at each node, whether early exercise is preferable to holding the option for a further time period:

 $$
 V^X(t_k)=B(t_k)\cdot\max\Bigg\{\frac{X(t_k)}{B(t_k)},\mathbb{E}^Q\Bigg[\frac{V^X(t_{k+1})}{B(t_{k+1})}\ \Bigg\vert\  \mathcal{P}_{t_k}\Bigg]\Bigg\}
 $$

The procedure outlined is identical for all lattice frameworks. Still, the way such procedure is implemented depends, at all steps, on the choice of the model.

### Pricing with the Willow Tree
__Lattice for the stock price.__ The willow tree models the stock price process, $\big\{S_{t_k}\big\}_{k=0}^n$, as a function of $Y_{t_k}$, the discrete Markov process. Each realisation of the process, $S_{ik}$, is defined, for $i=1,...,m$, and for $k=0,...,n$, as:

$$
S_{ik}=S_0\exp\bigg\{\Big(r-\frac{1}{2}\sigma^2\Big)t_k+\sigma z_i\sqrt{t_k}\bigg\}
$$

In the equation, $\big\{t_k\big\}_{k=0}^n$, such that $0=t_0<t_1<\dots<t_n=T$, is the set of elements of the $(n+1)$-valued vector $\mathbf{t}$, and $z_i\sqrt{t_k}$, the $(i,k)$-th element of grid $\mathbf{G}$ (both are defined in [Mathematical Development of the Willow Tree](02.ipynb)).

The set of stock price realisations, $\big\{S_{ik}\big\}$, can be represented on a grid:

$$
\mathbf{S}=
\begin{bmatrix}
S_0 & S_{11} & \dots & S_{1T}\\
\vdots & \vdots & \ddots & \vdots\\
S_0 & S_{m1} & \dots & S_{mT}
\end{bmatrix}
$$

The first index, $i$, denotes the position of the stock price realisation along the standard normal distribution, while the second one, $k$, its position in time. Because both $t_0$ and $Y_{t_0}$ are null, all the prices in the first column of $\mathbf{S}$ are identical, and equal to $S_0$. The content of matrix $\mathbf{S}$ can be expressed with a very succinct formula (a matrix at the exponent denotes element-wise multiplication):

$$
\mathbf{S}=S_0\exp{\bigg\{\Big(r-\frac{1}{2}\sigma^2\Big)\mathbf{T}+\sigma\mathbf{G}\bigg\}}
$$

in which $\mathbf{T}$ is an $m\times n$ matrix obtained by expanding row vector $\mathbf{t}$ in the column dimension of $\mathbf{G}$:

$$
\mathbf{T}=
\begin{bmatrix}
0 & t_1 & t_2 & \dots & T\\
\vdots &  \vdots &\vdots & \ddots & \vdots\\
0 & t_1 & t_2 & \dots & T
\end{bmatrix}
$$

The equation for $\mathbf{S}$ denotes the forward part of the algorithm. It starts from $S_0$, the stock price at valuation date, and generates the paths of the process as a function of $\big\{z_i\big\}_{i=1}^m$ and $\big\{t_k\big\}_{k=0}^n$.

__Payoffs at maturity.__ The payoffs at maturity, $\big\{X_i^T\big\}_{i=1}^m$, are a mapping of the stock prices in the last column of matrix $\mathbf{S}$:

$$
\begin{align*}
\begin{bmatrix}
S_{1T}& S_{2T} & \dots & S_{mT}
\end{bmatrix}
^{\text{T}}
\quad
\Longrightarrow
\quad
\begin{bmatrix}
X_1^T & X_2^T & \dots & X_m^T
\end{bmatrix}
^{\text{T}}
\end{align*}
$$

__Price at valuation date.__ At expiry date, $t_n=T$, the $m$ possible values of the instrument, $V^X_i$, coincide with the realised payoffs:

$$
V^X_i(T)=X_i^T,\qquad i=1,...,m
$$

At each previous date, $t_{n-1},...,t_0$, and for all nodes, the values depend on the kind of derivative. For European-type derivatives, it suffices to compute the conditional expectation of the discounted future payoffs with respect to the sets of transition probabilities for the period:

$$
V_i^X\big(t_k\big)=e^{-r(t_{k+1}-t_k)}\sum_{j=1}^mp_{ij}^kV^X_j\big(t_{k+1}\big)
$$

with $\mathbf{P}^{(k)}=\big[p_{ij}^k\big]$ the transition matrix for time interval $(t_k,t_{k+1})$. At time $t_0=0$, the value of the derivative is:

$$
V^X(0)=e^{-rt_1} \sum_{j=1}^mq_jV_j^X(t_1)
$$

From the stationarity requirement (see [The Linear Programming Problem](03.ipynb)) comes a formula of paramount importance [4], [8]:

$$
V^X(0) = \exp\bigg\{-r\sum_{k=0}^{n-1}\big(t_{k+1}-t_k\big)\bigg\}\ \mathbf{q}^{\text{T}}\prod_{k=1}^{n-1}\mathbf{P}^{(k)}\ V^X(T)=e^{-rT}\mathbf{q}^{\text{T}}\ V^X(T)
$$

Hence, if the transition matrices reproduce a stationary distribution at each time step, it is possible to let go of matrix product, and directly multiply $\mathbf{q}$ by the vector of discounted option payoffs.

For American-type derivative instruments, at each previous date, $t_{n-1},...,t_0$, the algorithm becomes:

$$
V^X_i\big(t_k\big)=\max\bigg\{X_i^{t_k},\ e^{-r(t_{k+1}-t_k)}\sum_{j=1}^mp_{ij}^kV^X_j\big(t_{k+1}\big)\bigg\}
$$

$X_i^{t_k}$, the immediate payoff of the option in node $i$ at time $t_k$, is the $(i,k)$-th element of $\mathbf{X}$, the matrix found by mapping all columns of $\mathbf{S}$ to $\phi$:

$$
\mathbf{X}=
\begin{bmatrix}
X^0 & X_1^1 & \dots & X_1^n\\
\vdots & \vdots & \ddots & \vdots\\
X^0 & X^1_m & \dots & X^n_m
\end{bmatrix}
$$

The value at time $t_0=0$ is computed as above.

Previous: [Xu, Hong, and Qin's Methodology](06.ipynb) | [Table of Contents](00.ipynb) | Next: [The Willow Tree: A Brief Recap](08.ipynb)